# Kaggle Air Quality Prediction: EDA
---
Goal of the notebook: EDA

Inputs of the notebook:

Output of the notebook:


Takeaways:

Questions, thoughts and remarks:

## Imports

In [2]:
import pandas as pd
import numpy as np

%load_ext autoreload
%autoreload 2

## 1. Data

In [3]:
df = pd.read_csv("../data/train.csv")
df

,id,valeur_NO2,valeur_CO,valeur_O3,valeur_PM10,valeur_PM25
0,2020-01-01 00,42.9,0.718,15.7,73.1,64.4
1,2020-01-01 01,33.6,0.587,10.1,74.8,66.0
2,2020-01-01 02,29.3,NaN,5.1,51.0,44.9
3,2020-01-01 03,30.5,0.246,7.2,27.7,25.1
4,2020-01-01 04,29.3,0.204,8.3,15.3,13.6
...,...,...,...,...,...,...
40986,2024-09-03 18,NaN,0.222,55.1,12.0,5.3
40987,2024-09-03 19,NaN,0.245,48.2,13.4,7.0
40988,2024-09-03 20,NaN,0.234,44.5,12.4,7.1
40989,2024-09-03 21,NaN,0.225,25.9,10.6,5.4
